## Linear Program with Cost and Generation Profile Dataframes

This linear program builds on the simple lp code by 1) pulling resource attributes and costs from a dataframe and 2) building out the solar generation profile to include other nondispatchable resources.

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting library
import datetime as dt
import pandas as pd

from ortools.linear_solver import pywraplp

In [2]:
import utils

In [3]:
resourcedf_solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Introduce objective object so we can refer to it in the for loop.
objective = resourcedf_solver.Objective()

In [4]:
# Load Harbor historical hourly generation and emissions for 2014-2018. Group by datetime to sum generation and emissions from all units. Filter for specific year -- make sure selected year isn't leap year.
harborgen = utils.get_harbor_data('data/HarborHourly_2014-18.csv')
harborgen = harborgen.groupby(['datetime'])['mwh'].sum()

# Create string object that is the chosen year
year = '2018'

# Filter Harbor data by the chosen year, using the string object above.
harborgen = harborgen.filter(like=year, axis=0)

In [5]:
# Load generation profiles for nondispatchable resources (mwh generated each hour by 1 KW of capacity).
profiles = pd.read_csv('data/gen_profiles.csv')

# Match profiles index to harborgen index. **Currently this only works for one year -- for multiple years, need to repeat gen profiles.
profiles.insert(0, 'datetime', harborgen.index)
profiles = profiles.set_index('datetime')

In [6]:
resources = pd.read_csv('data/resource_costs.csv')
resources = resources.set_index('resource')

In [7]:
resources

,legacy,existing_mw,dispatchable,capex,fixed,variable,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
resource,,,,,,,,,,,,,
gas_harbor,y,450.0,y,450.0,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
outofbasin,y,3000.0,y,3000.0,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gas_repower,n,NaN,y,5.0,NaN,0.10,0.06,0.06,0.06,0.06,0.06,0.06,0.36
solar,n,NaN,n,6.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
demand_response,n,NaN,y,4.0,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storage_utility,n,NaN,y,2.0,NaN,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storage_res,n,NaN,y,1.0,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storage_ci,n,NaN,y,2.0,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storage_diesel,n,NaN,y,3.0,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Declare nameplate capacity variables for each resource in resource cost dataframe.
capacity_vars = {}
for resource in resources.index:
    capacity = resourcedf_solver.NumVar(0, resourcedf_solver.infinity(), str(resource))
    capacity_vars[resource] = capacity
    
capacity_vars

{'gas_harbor': gas_harbor,
 'outofbasin': outofbasin,
 'gas_repower': gas_repower,
 'solar': solar,
 'demand_response': demand_response,
 'storage_utility': storage_utility,
 'storage_res': storage_res,
 'storage_ci': storage_ci,
 'storage_diesel': storage_diesel,
 'ee_1': ee_1,
 'ee_2': ee_2,
 'ee_3': ee_3,
 'ee_4': ee_4,
 'ee_5': ee_5,
 'ee_6': ee_6,
 'ee_7': ee_7,
 'ee_8': ee_8,
 'ee_9': ee_9,
 'ee_10': ee_10}

In [9]:
#Create filtered dataframes for dispatchable and nondispatchable resources.
disp = resources.loc[resources['dispatchable'] == 'y']
nondisp = resources.loc[resources['dispatchable'] == 'n']

In [10]:
#Create a dictionary to hold a list for each dispatchable resource that keeps track of its hourly generation variables.
disp_gen = {}
for resource in disp.index:
    disp_gen[resource] = []

In [11]:
disp_gen

{'gas_harbor': [],
 'outofbasin': [],
 'gas_repower': [],
 'demand_response': [],
 'storage_utility': [],
 'storage_res': [],
 'storage_ci': [],
 'storage_diesel': []}

In [12]:
nondisp

,legacy,existing_mw,dispatchable,capex,fixed,variable,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
resource,,,,,,,,,,,,,
solar,n,NaN,n,6.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_1,n,NaN,n,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_2,n,NaN,n,4.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_3,n,NaN,n,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_4,n,NaN,n,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_5,n,NaN,n,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_6,n,NaN,n,6.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_7,n,NaN,n,7.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee_8,n,NaN,n,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
for ind in harborgen.index:
    a = harborgen.loc[ind]
    if a != 0:
        print(a, ind)
        break

6.84 2018-01-03 07:00:00


In [14]:
for resource in disp.index:
    print(disp.loc[resource,'variable'])

0.21
0.03
0.1
0.15
0.05
0.06
0.06
0.06


In [15]:
# Loop through every hour, creating 1) hourly generation variables for each dispatchable resource, 
# 2) hourly constraints, and 3) adding variable cost coefficients to each hourly generation variable.
for ind in harborgen.index:
    
    #Summed generation from all resources must be equal or greater to demand in all hours.
    fulfill_demand = resourcedf_solver.Constraint(harborgen.loc[ind], resourcedf_solver.infinity(), '_demand'+str(ind))
    
    #Create generation variable for each dispatchable resource for every hour. Append hourly gen variable to the list for that resource, located in the disp_gen dictionary.
    #Create constraint that generation must be less than or equal to capacity for each dispatchable resource for all hours.
    for resource in disp.index:
        
        max_gen = resourcedf_solver.Constraint(0, resourcedf_solver.infinity(), '_max_gen_'+resource+'_' + str(ind))
        gen = resourcedf_solver.NumVar(0, resourcedf_solver.infinity(), '_gen'+ str(ind))
        disp_gen[resource].append(gen)
        objective.SetCoefficient(gen, disp.loc[resource,'variable'])
        
        #Set coefficients for the hourly gen variables for the fulfill_demand constraint.
        fulfill_demand.SetCoefficient(gen, 1)
        
        #Set coefficients for dispatchable capacity variables and hourly gen variables for the max_gen = capacity constraint. 
        capacity = capacity_vars[resource]
        max_gen.SetCoefficient(capacity, 1)
        max_gen.SetCoefficient(gen, -1)
    
    #For each nondispatchable resource, set the coefficient of the capacity variable to its generation profile scaling factor. **Make sure units are aligned here (kw vs. mw capacities)
    for resource in nondisp.index: 
        capacity = capacity_vars[resource]
        coefficient = profiles.loc[ind, resource]
        fulfill_demand.SetCoefficient(capacity, coefficient)


In [23]:
## TODO: Add in fixed costs -- we need to assume a certain plant lifetime and amortize over that period, using discount rate. ******
for resource in resources.index:
    capex = resources.loc[resource, 'capex']
    objective.SetCoefficient(capacity_vars[resource], capex)

objective.SetMinimization()
status = resourcedf_solver.Solve()
print("total cost =", objective.Value())

for resource in capacity_vars:
    print(str(capacity_vars[resource]) + ' capacity =' + str(capacity_vars[resource].solution_value()))

# Sum generation for each resource and print.
for resource in disp.index:
    summed_gen = 0
    for i_gen in disp_gen[str(resource)]:
        summed_gen += i_gen.solution_value()
    print(str(resource) + ': annual generation =' + str(summed_gen))
    
## Sum annual generation for nondispatchable resources.
summed_gen = profiles.sum()

for resource in nondisp.index:
    summed_gen.filter(resource)
    capacity = capacity_vars[resource].solution_value()
    gen = summed_gen[resource] * capacity
    print(str(resource) + ': annual generation =' + str(gen))

total cost = 1831.4494142857143
gas_harbor capacity =0.0
outofbasin capacity =0.0
gas_repower capacity =0.0
solar capacity =0.0
demand_response capacity =0.0
storage_utility capacity =82.0
storage_res capacity =143.0
storage_ci capacity =0.0
storage_diesel capacity =0.0
ee_1 capacity =0.0
ee_2 capacity =0.0
ee_3 capacity =0.0
ee_4 capacity =0.0
ee_5 capacity =428.57142857142856
ee_6 capacity =0.0
ee_7 capacity =0.0
ee_8 capacity =0.0
ee_9 capacity =0.0
ee_10 capacity =0.0
gas_harbor: annual generation =0.0
outofbasin: annual generation =0.0
gas_repower: annual generation =0.0
demand_response: annual generation =0.0
storage_utility: annual generation =11632.935714285708
storage_res: annual generation =8570.52
storage_ci: annual generation =0.0
storage_diesel: annual generation =0.0
solar: annual generation =0.0
ee_1: annual generation =0.0
ee_2: annual generation =0.0
ee_3: annual generation =0.0
ee_4: annual generation =0.0
ee_5: annual generation =744012.8571428541
ee_6: annual genera

In [41]:
first = resourcedf_solver.constraints()[0]